<a href="https://colab.research.google.com/github/mangeshb1/Gen_AI_And_NLP_Experiments/blob/main/Gen_AI_And_Chatbots/RAG_Open_AI_AstraVDB_And_Streamlit_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build Your Own RAG using RAGStack
This notebook shows the steps to take to use the Astra DB Vector Store as a means to make LLM interactions meaningfull and without hallucinations. The approach taken here is Retrieval Augmented Generation.

1. About the content in a CNN dataset
2. How to interact with the OpenAI Chat Model *without* providing this context
3. How to load this context into Astra DB Vector Store
4. How to run a semantic similarity search on Astra DB Vector Store
5. How to use this context *with* the OpenAI Chat Model

## Install dependencies

In [1]:
!pip install ragstack-ai python-dotenv pandas datasets pipdeptree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## Visualize Ragstack dependencies
RAGStack is a curated stack of the best open-source software for easing implementation of the RAG pattern in production-ready applications using Astra Vector DB or Apache Cassandra as a vector store.

A single command (pip install ragstack-ai) unlocks all the open-source packages required to build production-ready RAG applications with LangChain and the Astra Vector database.

For each open-source project included in RAGStack, we select a version lineup and then test the combination for compatibility, performance, and security. Our extensive test suite ensures that RAGStack components work well together so you can confidently deploy them in production. We also run security scans on all components using industry-standard tools to ensure that you are not exposed to known vulnerabilities.

In [2]:
!pipdeptree -p ragstack-ai

Warning!!! Possibly conflicting dependencies found:
* ipython==7.34.0
 - jedi [required: >=0.16, installed: ?]
* PyGObject==3.42.1
 - pycairo [required: >=1.16.0, installed: ?]
------------------------------------------------------------------------
ragstack-ai==0.8.0
├── astrapy [required: >=0.7.0,<0.8.0, installed: 0.7.6]
│   ├── cassio [required: >=0.1.4,<0.2.0, installed: 0.1.5]
│   │   ├── cassandra-driver [required: >=3.28.0, installed: 3.29.0]
│   │   │   └── geomet [required: >=0.1,<0.3, installed: 0.2.1.post1]
│   │   │       ├── click [required: Any, installed: 8.1.7]
│   │   │       └── six [required: Any, installed: 1.16.0]
│   │   ├── numpy [required: >=1.0, installed: 1.25.2]
│   │   └── requests [required: >=2, installed: 2.31.0]
│   │       ├── certifi [required: >=2017.4.17, installed: 2024.2.2]
│   │       ├── charset-normalizer [required: >=2,<4, installed: 3.3.2]
│   │       ├── idna [required: >=2.5,<4, installed: 3.6]
│   │       └── urllib3 [required: >=1.21.1,<3

## Provide secrets for Astra DB and OpenAI
Make sure to save your secrets somewhere safe and do not share it with anyone. Follow the below steps and provide the **Astra DB API Endpoint**, **Astra DB ApplicationToken** and **OpenAI API Key**.
### Sign up for Astra DB
Make sure you have a vector-capable Astra database (get one for free at [astra.datastax.com](https://astra.datastax.com))
- You will be asked to provide the **API Endpoint** which can be found in the right pane underneath *Database details*.
- Ensure you have an **Application Token** for your database which can be created in the right pane underneath *Database details*.

### Sign up for OpenAI
- Create an [OpenAI account](https://platform.openai.com/signup) or [sign in](https://platform.openai.com/login).
- Navigate to the [API key page](https://platform.openai.com/account/api-keys) and create a new **Secret Key**, optionally naming the key.


In [3]:
from getpass import getpass

astra_endpoint = getpass('Your Astra DB API Endpoint: ')
astra_token = getpass('Your Astra DB Application Token: ')
openai_api_key = getpass('Your OpenAI API Key: ')

Your Astra DB API Endpoint: ··········
Your Astra DB Application Token: ··········
Your OpenAI API Key: ··········


In [5]:
openai_api_key = getpass('Your OpenAI API Key: ')

Your OpenAI API Key: ··········


## Call Open AI's Chat Model
In this example we'll ask what Daniell Radcliffe recieves when he turns 18.

As Open AI has no access to the CNN documents, it will come up with some answer that makes no sense.

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

template = """
You are a philosopher that draws inspiration from great thinkers of the past
to craft well-thought answers to user questions. Use the provided context as the basis
for your answers and do not make up new reasoning paths - just mix-and-match what you are given.
Your answers must be extensively written.

QUESTION: {question}

YOUR ANSWER:"""
prompt = ChatPromptTemplate.from_messages([("system", template)])

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    temperature=0.3,
    model='gpt-3.5-turbo')

inputs = RunnableMap({
  'question': lambda x: x['question']
})
chain = inputs | prompt | llm | StrOutputParser()

chain.invoke({"question": "What kind of fortune does Daniel Radcliffe get when he turns 18?"})

'When Daniel Radcliffe turns 18, he receives a fortune that is not measured in material wealth but in the richness of experience and opportunity that comes with newfound adulthood. Just as the ancient Greek philosopher Aristotle believed that true happiness comes from living a life of virtue and fulfilling one\'s potential, Radcliffe\'s coming of age signifies a transition into a new chapter of his life where he can explore his talents and passions more freely.\n\nIn the words of the Roman Stoic philosopher Seneca, "Luck is what happens when preparation meets opportunity." As Radcliffe enters adulthood, he is presented with a plethora of opportunities to further hone his craft as an actor, expand his horizons beyond the wizarding world of Harry Potter, and make a meaningful impact in the entertainment industry. This fortune is not handed to him on a silver platter but is a result of his dedication, hard work, and the support of those around him.\n\nTurning 18 is a symbolic milestone th

## Load data from CNN

In [7]:
import datasets

def load_articles(n=5):
  dataset = datasets.load_dataset('cnn_dailymail', '3.0.0', split='train', streaming=True)
  data = dataset.take(n)
  return [d['article']
          for d in data]

articles = load_articles()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Check out some content
In this example we can read that when Daniel Radcliffe turns 18, he'll gain access to £20 million.

In [8]:
print(articles[0])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

## Generate chunks to load into the Vector Store
Now let's load the CNN data into the Astra DB Vector Store.
1. First we'll chunk up the data so that it can be loaded in multiple pieces.
2. Then we'll create a new Vector Store on Astra DB.
3. Lastly, we'll load up the documents. As part of this step, the data will be vectorized and it's embeddings stored in the Vector Store.

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

documents = splitter.create_documents(articles)
document_chunks = splitter.split_documents(documents)

print(document_chunks[0])

page_content='LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import AstraDB

# Create a new Astra DB Vector Store
vector_store = AstraDB(
    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key),
    collection_name="my_store",
    api_endpoint=astra_endpoint,
    token=astra_token
)

In [11]:
# Load the CNN documents into the Astra DB Vector Store (Only the first time)
vector_store.add_documents(document_chunks)

['72a1aa91deca42e4b4643c73f72b6feb',
 '308aee7fd2a24b6fa5234c0bb418ab93',
 '2020d2a0dff343f985576d1bb0871627',
 '82b9e33dfff9442ca0e637764e35c444',
 '8301975d99474d72a75c434236f4ffcd',
 '65a7096d08404725b154848d420c9d91',
 '544be33612524ba092480ed632dac4ba',
 '4d8a8df7f5a64c008c28561bc4b00116',
 'cc2b446f69f24227bd37e60ad45b2a60',
 '970f84423c134ea69488222a9c36a5ed',
 '65cf5a1c3e6b40cf91f1717d9fd42b95',
 'aa27b6da4d48400398dc47baa540fc6e',
 '4e9712ed1cfe467fb583e33bff96e5e3',
 '7c00335df06444de9f4595d6846f2c84',
 '2202563114fb4f38ac5f105021db29b0',
 'a53f7300c93e4fc99d89e8e24204566c',
 'a8d5ee4c92b3449bab8ba3820ab3bb3b',
 '27497cf00def409da81c6e617ed5cbd2',
 '95361130547e4336998cd042297f03d8',
 '25f1ffe2c83446b49888c836f57469ca',
 '9c5a04281f5648729eb0c61b9a61342d']

## Run a semantic query on the Astra DB Vector Store
Here you'll see that Astra DB retrieves relevant documents given the query.

In [12]:
query = 'What kind of fortune does Daniel Radcliffe get when he turns 18?'
vector_store.similarity_search(query, k=2)

[Document(page_content='LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box 

## Call OpenAI's Chat Model again
Now let's run the query again on the OpenAI Chat Model while inserting the relevant context from the Astra DB Vector Store to make the response meaningfull and stop hallucinating.

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

# Get the retriever for the Chat Model
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5}
)

# Create the prompt template
template = """
You are a philosopher that draws inspiration from great thinkers of the past
to craft well-thought answers to user questions. Use the provided context as the basis
for your answers and do not make up new reasoning paths - just mix-and-match what you are given.
Your answers must be extensively written.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""
prompt = ChatPromptTemplate.from_messages([("system", template)])

# Define the chain
inputs = RunnableMap({
  'context': lambda x: retriever.get_relevant_documents(x['question']),
  'question': lambda x: x['question']
})
chain = inputs | prompt | llm | StrOutputParser()

# Call the chain with the question
chain.invoke({"question": "What kind of fortune does Daniel Radcliffe get when he turns 18?"})

'When Daniel Radcliffe turned 18, he gained access to a reported £20 million ($41.1 million) fortune. This substantial sum of money, accumulated from his earnings from the first five Harry Potter films, was held in a trust fund that he had not been able to touch until he reached this milestone. Despite the significant wealth at his disposal, Radcliffe expressed a grounded attitude towards his newfound fortune, stating that he did not plan to indulge in extravagant purchases like a massive sports car collection or lavish parties. Instead, he mentioned a preference for modest expenses such as books, CDs, and DVDs that cost around £10. This responsible approach to his wealth reflects his maturity and level-headedness in handling financial matters.'

Let's add a chat interface
In this part of the demo, we'll actually create a fully working chatbot using Streamlit!

In [14]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


## Install the local tunnel to view the webpage

In [15]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.226s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


# Create the Chatbot

In [16]:
%%writefile app.py

import streamlit as st
import tempfile, os
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import AstraDB
from langchain.schema.runnable import RunnableMap
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Streaming call back handler for responses
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs):
        self.text += token
        self.container.markdown(self.text + "▌")

# Function for Vectorizing uploaded data into Astra DB
def vectorize_text(uploaded_files, vector_store):
    for uploaded_file in uploaded_files:
        if uploaded_file is not None:

            # Write to temporary file
            temp_dir = tempfile.TemporaryDirectory()
            file = uploaded_file
            print(f"""Processing: {file}""")
            temp_filepath = os.path.join(temp_dir.name, file.name)
            with open(temp_filepath, 'wb') as f:
                f.write(file.getvalue())

            # Process TXT
            if uploaded_file.name.endswith('txt'):
                file = [uploaded_file.read().decode()]

                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 1500,
                    chunk_overlap  = 100
                )

                texts = text_splitter.create_documents(file, [{'source': uploaded_file.name}])
                vector_store.add_documents(texts)
                st.info(f"Loaded {len(texts)} chunks")

# Cache prompt for future runs
@st.cache_data()
def load_prompt():
    template = """You're a helpful AI assistent tasked to answer the user's questions.
You're friendly and you answer extensively with multiple sentences. You prefer to use bulletpoints to summarize.

CONTEXT:
{context}

QUESTION:
{question}

YOUR ANSWER:"""
    return ChatPromptTemplate.from_messages([("system", template)])

# Cache OpenAI Chat Model for future runs
@st.cache_resource()
def load_chat_model(openai_api_key):
    return ChatOpenAI(
        openai_api_key=openai_api_key,
        temperature=0.3,
        model='gpt-3.5-turbo',
        streaming=True,
        verbose=True
    )

# Cache the Astra DB Vector Store for future runs
@st.cache_resource(show_spinner='Connecting to Astra DB Vector Store')
def load_vector_store(_astra_db_endpoint, astra_db_secret, openai_api_key):
    # Connect to the Vector Store
    vector_store = AstraDB(
        embedding=OpenAIEmbeddings(openai_api_key=openai_api_key),
        collection_name="my_store",
        api_endpoint=astra_db_endpoint,
        token=astra_db_secret
    )
    return vector_store

# Cache the Retriever for future runs
@st.cache_resource(show_spinner='Getting retriever')
def load_retriever(_vector_store):
    # Get the retriever for the Chat Model
    retriever = vector_store.as_retriever(
        search_kwargs={"k": 5}
    )
    return retriever

# Start with empty messages, stored in session state
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Draw a title and some markdown
st.title("Your personal Efficiency Booster")
st.markdown("""Generative AI is considered to bring the next Industrial Revolution.
Why? Studies show a **37% efficiency boost** in day to day work activities!""")

# Get the secrets
astra_db_endpoint = st.sidebar.text_input('Astra DB Endpoint', type="password")
astra_db_secret = st.sidebar.text_input('Astra DB Secret', type="password")
openai_api_key = st.sidebar.text_input('OpenAI API Key', type="password")

# Draw all messages, both user and bot so far (every time the app reruns)
for message in st.session_state.messages:
    st.chat_message(message['role']).markdown(message['content'])

# Draw the chat input box
if not openai_api_key.startswith('sk-') or not astra_db_endpoint.startswith('https') or not astra_db_secret.startswith('AstraCS'):
    st.warning('Please enter your Astra DB Endpoint, Astra DB Secret and Open AI API Key!', icon='⚠')

else:
    prompt = load_prompt()
    chat_model = load_chat_model(openai_api_key)
    vector_store = load_vector_store(astra_db_endpoint, astra_db_secret, openai_api_key)
    retriever = load_retriever(vector_store)

    # Include the upload form for new data to be Vectorized
    with st.sidebar:
        st.divider()
        uploaded_file = st.file_uploader('Upload a document for additional context', type=['txt'], accept_multiple_files=True)
        submitted = st.button('Save to Astra DB')
        if submitted:
            vectorize_text(uploaded_file, vector_store)

    if question := st.chat_input("What's up?"):
            # Store the user's question in a session object for redrawing next time
            st.session_state.messages.append({"role": "human", "content": question})

            # Draw the user's question
            with st.chat_message('human'):
                st.markdown(question)

            # UI placeholder to start filling with agent response
            with st.chat_message('assistant'):
                response_placeholder = st.empty()

            # Generate the answer by calling OpenAI's Chat Model
            inputs = RunnableMap({
                'context': lambda x: retriever.get_relevant_documents(x['question']),
                'question': lambda x: x['question']
            })
            chain = inputs | prompt | chat_model
            response = chain.invoke({'question': question}, config={'callbacks': [StreamHandler(response_placeholder)]})
            answer = response.content

            # Store the bot's answer in a session object for redrawing next time
            st.session_state.messages.append({"role": "ai", "content": answer})

            # Write the final answer without the cursor
            response_placeholder.markdown(answer)

Writing app.py


# Now run the Chatbot!
We have to know the public URL of the server as a password.
Once we know that, we can kick off the Streamlit app through the tunnel.

In [17]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.251.20


In [18]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 3.312s
your url is: https://upset-pugs-push.loca.lt
^C
